# Red Wine Quality Project
***
***

# Goal
***
- Predict the quality of red wines based on some of their various physiochemical attributes

# Setup
***

In [33]:
# establishing environment
import sklearn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE

import warnings
warnings.filterwarnings("ignore")

# Acquire
Acquiring the data for this project
***

In [34]:
wines = pd.read_csv('winequality-red.csv')

wines.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


# Prepare
Preparing the data for this project

## Splitting data

In [35]:
# splitting data
train_validate, test = train_test_split(wines, test_size=.2, random_state=123)
train, validate = train_test_split(train_validate, test_size=.3, random_state=123)

## Scaling data

In [36]:
# identifying columns to scale
col_to_scale = []

for col in wines:
    if (wines[col].min() < 0 or wines[col].max() > 1) and col != 'quality':
        col_to_scale.append(col)

col_to_scale

['fixed acidity',
 'volatile acidity',
 'residual sugar',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']

In [47]:
# creating scaler object
scaler = sklearn.preprocessing.MinMaxScaler()

# creating list of dfs to iterate upon
wine_dfs = [train, validate, test]

# creating cols that will hold scaled values
for df in wine_dfs:
    for col in col_to_scale:
        train[col + '_s'] = 0
        validate[col + '_s'] = 0
        test[col + '_s'] = 0

# fitting scaler to train column and scaling after
#train[col_scaled_names] = scaler.fit_transform(train[col_to_scale])

for df in wine_dfs:
    if df == 'train':
        for col in df:
            if col[-2:] == '_s':
                df[col] = scaler.fit_transform(df[col])
    if df != 'train':
        for col in df:
            if col[-2:] == '_s':
                df[col] = scaler.transform(df[col])

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [51]:
for df in wine_dfs:
    print(df.values)
    #if df == 'train':

[[8.3  0.34 0.4  ... 0.   0.   0.  ]
 [6.7  0.54 0.13 ... 0.   0.   0.  ]
 [8.2  0.28 0.6  ... 0.   0.   0.  ]
 ...
 [7.6  0.42 0.25 ... 0.   0.   0.  ]
 [8.2  0.44 0.24 ... 0.   0.   0.  ]
 [8.7  0.84 0.   ... 0.   0.   0.  ]]
[[ 6.9    0.58   0.01  ...  0.     0.     0.   ]
 [ 9.4    0.59   0.14  ...  0.     0.     0.   ]
 [ 7.9    0.5    0.33  ...  0.     0.     0.   ]
 ...
 [ 7.6    0.41   0.24  ...  0.     0.     0.   ]
 [10.     0.38   0.38  ...  0.     0.     0.   ]
 [ 7.5    1.115  0.1   ...  0.     0.     0.   ]]
[[10.     0.46   0.44  ...  0.     0.     0.   ]
 [ 9.5    0.57   0.27  ...  0.     0.     0.   ]
 [ 7.3    0.91   0.1   ...  0.     0.     0.   ]
 ...
 [ 6.6    0.44   0.09  ...  0.     0.     0.   ]
 [ 6.7    0.64   0.23  ...  0.     0.     0.   ]
 [ 6.6    0.725  0.2   ...  0.     0.     0.   ]]
